In [3]:
import numpy as np

In [4]:
class Operation():
    
    def __init__(self,input_nodes=[]):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
    
    def compute(self):
        pass

In [5]:
class add(Operation):
    
    def __init__(self,x,y):
        
        super().__init__(([x,y]))
        
    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        
        return x_var + y_var

In [6]:
class multiply(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        
        return x_var * y_var

In [7]:
class matmul(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        
        return x_var.dot(y_var)

In [8]:
class Placeholder():
    
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [9]:
class Variable():
    
    def __init__(self,initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [10]:
class Graph():
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        
        global _default_graph
        _default_graph = self
        print("Variable",self.variables)
        print("Operations",self.operations)
        print("Placeholder",self.placeholders)

z = Ax + b

A = 10

b = 1

z = 10x + 1

In [11]:
g = Graph()

In [12]:
g.set_as_default()

Variable []
Operations []
Placeholder []


In [13]:
A = Variable(10)

In [14]:
g.set_as_default()

Variable [<__main__.Variable object at 0x10ffd6400>]
Operations []
Placeholder []


In [15]:
b = Variable(1)

In [16]:
g.set_as_default()

Variable [<__main__.Variable object at 0x10ffd6400>, <__main__.Variable object at 0x10ffd69e8>]
Operations []
Placeholder []


In [17]:
x = Placeholder()

In [18]:
y = multiply(A,x)

In [19]:
z = add(y,b)

In [20]:
g.set_as_default()

Variable [<__main__.Variable object at 0x10ffd6400>, <__main__.Variable object at 0x10ffd69e8>]
Operations [<__main__.multiply object at 0x10ffe0160>, <__main__.add object at 0x10ffe0400>]
Placeholder [<__main__.Placeholder object at 0x10ffd6e80>]


In [21]:
def traverse_postorder(operation):
    
    nodes_postorder = []
    
    def recurse(node):
        if isinstance(node,Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
        
    recurse(operation)
    return nodes_postorder

In [22]:
class Session():
    
    def run(self,operation,feed_dict={}):
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            print(node)
            if type(node) == Placeholder:
                node.output = feed_dict[node]
                print("Node Ouput :",node.output)
            
            elif type(node) == Variable:
                node.output = node.value
                print("Node Value :",node.value)
                
            else:
                #Operation
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)
                print("Node input node : ",node.input_nodes)
                
            if type(node.output) == list:
                node.output = np.array(node.output)
                
                
        return operation.output
                
                
                
                
                

In [23]:
sess = Session()

In [24]:
result = sess.run(operation=z,feed_dict={x:10})

Node Value : 10
Node Ouput : 10
Node input node :  [<__main__.Variable object at 0x10ffd6400>, <__main__.Placeholder object at 0x10ffd6e80>]
Node Value : 1
Node input node :  [<__main__.multiply object at 0x10ffe0160>, <__main__.Variable object at 0x10ffd69e8>]


In [25]:
result

101

In [26]:
g = Graph()
g.set_as_default()

A = Variable([[10,20],[30,40]])

b = Variable([1,2,])

x = Placeholder()

y = matmul(A,x)

z = add(y,b)

Variable []
Operations []
Placeholder []


In [27]:
sess = Session()

In [28]:
sess.run(operation=z,feed_dict={x:10})

Node Value : [[10, 20], [30, 40]]
Node Ouput : 10
Node input node :  [<__main__.Variable object at 0x10ffe05f8>, <__main__.Placeholder object at 0x1080596a0>]
Node Value : [1, 2]
Node input node :  [<__main__.matmul object at 0x10ffc7b38>, <__main__.Variable object at 0x108059a58>]


array([[101, 202],
       [301, 402]])